Saving Grapes
Andrew Liu, Yilun Yin, Taylor Wang
November 13th

Summary

Part I: Introductin
The Spotted Lanternfly (SLF), an invasive species causing significant damage to vineyards, poses a growing threat to the viticulture industry worldwide. In vineyards, the primary method of combating SLF infestations is through the application of insecticides. However, this approach is fraught with challenges, including the timing of applications, environmental impact, and the potential effects on grape quality and vine health. In fact, indiscriminate use of insecticides can lead to delayed harvests, increased vulnerability to fungal diseases, and detrimental impacts on beneficial insect populations. This complexity necessitates a nuanced approach to insecticide use, particularly in balancing the immediate need to protect crops against long-term ecological and financial consequences. In this paper, we propose a dynamic and adaptive strategy for insecticide application in vineyards, aiming to optimize the timing and threshold for spraying based on a variety of factors such as SLF migration patterns, weather conditions, and the proximity to harvest time. The strategy is designed to minimize the negative impacts of SLF on vineyards while also considering the broader ecological implications and the sustainability of viticulture practices.

Part II: Data
1) Weather
2) Pesticide 

In [ ]:
import numpy as np
import pandas as pd

# PARAMS
temperature = 0
precipitation = 0

slf_pop = 1000
# To account for population growth on a yearly basis:
# new population = surviving population * (1 + proportion are females * egg masses laid per female * 35-45 eggs per mass) 

alpha = 0.5     # Proportion of lantern flies that are adults
# Since there is only one generation of lantern flies each year, assume that alpha
# is initially 0 from March to July. From July to August/September, we can increase alpha 
# using maybe softmax. From August/September to October/November, assume alpha = 1 (we only have adults)

adult_pop = alpha * slf_pop
nymph_pop = (1 - alpha) * slf_pop

pesticide_use = {}

In [6]:
def slf_population_model(temperature, precipitation, start_day=1, end_day=183):
    """
    Estimates the proportion of nymphs and adult SLFs from early May (day 1) to late October (day 183).
    
    Parameters:
    temperature (list): Daily average temperatures.
    precipitation (list): Daily precipitation levels.
    start_day (int): Start day of the period (default is 1, early May).
    end_day (int): End day of the period (default is 183, late October).
    
    Returns:
    dict: Proportions of nymphs and adults over time.
    """
    days = np.arange(start_day, end_day + 1)
    nymph_proportion = np.zeros_like(days, dtype=float)
    adult_proportion = np.zeros_like(days, dtype=float)

    # Simplified model based on general trends
    for i, day in enumerate(days):
        temp_factor = max(0, min(1, (temperature[i] - 10) / 15))  # Assuming optimal hatching around 25°C
        precip_factor = 1 - min(1, precipitation[i] / 10)  # Assuming less activity with high precipitation

        if day < 60:  # Early stage: mostly nymphs
            nymph_proportion[i] = temp_factor * precip_factor
        elif 60 <= day < 120:  # Transition period
            nymph_proportion[i] = (120 - day) / 60 * temp_factor * precip_factor
            adult_proportion[i] = (day - 60) / 60 * temp_factor * precip_factor
        else:  # Later stage: mostly adults
            adult_proportion[i] = temp_factor * precip_factor

    return {"days": days, "nymph_proportion": nymph_proportion, "adult_proportion": adult_proportion}

# Example usage
temperature = [25] * 183  # Constant temperature for simplicity
precipitation = [5] * 183  # Constant precipitation for simplicity
population = slf_population_model(temperature, precipitation)

# The 'population' dictionary contains the proportions of nymphs and adults for each day

The code makes a few key assumptions about the Spotted Lanternfly (SLF) life cycle and the impact of environmental factors, particularly temperature and precipitation:

Temperature Impact on Maturation:

The code assumes that higher temperatures, up to an optimal point, accelerate the maturation of SLFs from nymphs to adults. This is modeled by the temp_factor, which increases with temperature up to a certain threshold (assumed here as 25°C for optimal hatching). Beyond this threshold, the impact of temperature is not further increased in the model, which is a simplification.
The temperature effect is linearly scaled between a minimum threshold (10°C in this model) and the optimal temperature (25°C). Below 10°C, it's assumed there's no maturation (temp_factor = 0), and above 25°C, the maturation rate doesn't increase further (temp_factor = 1).
Precipitation Impact:

The model assumes that higher precipitation levels negatively impact the SLF's activity, including maturation. This is represented by precip_factor, which decreases as precipitation increases, assuming reduced activity in wet conditions.
The impact of precipitation is simplified and capped at a certain level (10mm in this model), beyond which it's assumed that additional precipitation doesn't further reduce SLF activity.
Life Cycle Timing:

The model divides the SLF life cycle into three phases based on the day of the year: early stage (mostly nymphs), transition period (mix of nymphs and adults), and later stage (mostly adults). These phases are set based on fixed day ranges, which is a simplification and may not accurately reflect variations in SLF development due to environmental or geographical factors.
Uniformity Across Population:

The model assumes uniformity in the response of the SLF population to environmental factors. In reality, there would be variations within the population, with some individuals developing faster or slower than others.
Constant Environmental Conditions:

In the example usage, constant temperature and precipitation are used for simplicity. However, in real-world scenarios, these factors would vary daily and would have a more complex impact on the SLF population.
This model is a basic representation and should ideally be refined with more detailed empirical data on the effects of temperature and precipitation on the SLF life cycle. The assumptions are made to provide a starting point for modeling, but they simplify the complex interactions in an actual ecological system.

https://extension.psu.edu/spotted-lanternfly-management-guide

"To protect pollinators that visit flowers for nectar, never spray any insecticide on plants that are blooming."
To reduce impact on other insects, we should consider reducing the application of pesticides while grapevines 
are flowering. Ideally, we use most effective insecticide with minimal PHI close to harvest date.

NEED TO FIND WHEN GRAPEVINE FLOWERING BEGINS AND ENDS

https://extension.psu.edu/spotted-lanternfly-management-in-vineyards

Table 1. shows that nymphs target grape plants starting in May to July, then adults target grapes from August to October.

"SLF are voracious feeders and can be extremely abundant as adults in vineyards. Adults start to appear in vineyards in August, but high populations are not typically observed until mid-to-late September (Figure 2). For vineyards that are first experiencing SLF, this phenology is typically shifted later into the season—you may not see large numbers invade the vineyard until October. After one or two years, adult SLF typically invade vineyards earlier in the season (late August)"

"More importantly, the majority of SLF adult population within a vineyard is observed on the edge; on average, 54 percent of the SLF population is within the first 50 feet of the vineyard edge. Depending on the landscape surrounding the vineyard, the edge of the vineyard may account for even higher SLF numbers (upward of 80 percent of the population)."

"There is one generation of SLF per year." "Egg masses usually contain around 35–45 eggs each (Figure 1A). A single SLF female can lay at least two egg masses."

"The majority of adult SLF observed in vineyards are female."

In [5]:
file_path = 'pesticides.csv'
df = pd.read_csv(file_path)

# Mapping of efficacy ratings to numerical values
efficacy_mapping = {
    'Excellent': 1,
    'Good': 0.75,
    'Poor': 0.5,
    'Variable': 0,  # Assuming 'Variable' means the effect is inconsistent
    'Not Recommended': 0,  # Assuming 'Not Recommended' means no effect
    '': None  # Assuming empty string means no data available
}

# Mapping the 'Effect on Adults' and 'Effect on Nymphs' columns in the DataFrame
df['Effect on Adults'] = df['Effect on Adults'].map(efficacy_mapping)
df['Effect on Nymphs'] = df['Effect on Nymphs'].map(efficacy_mapping)

# Defining the function to calculate insecticide effectiveness
def insecticide_effectiveness(df, day, nymph_population, adult_population):
    df['Effect_on_Nymphs'] = df['Effect on Nymphs'] * nymph_population
    df['Effect_on_Adults'] = df['Effect on Adults'] * adult_population
        
    # Assuming 'REI' stands for Re-Entry Interval, which is not present in the given data
    # Adding a placeholder for 'REI' in the DataFrame for the function to work
    # df['REI'] = some_value  # Replace some_value with actual REI data if available
    
    # The 'PHI (days)' column in the CSV is assumed to be the PHI value
    df['Available_in'] = df['PHI (days)'].apply(lambda x: max(0, x - day))
    df['Available_until_Harvest'] = df['PHI (days)'].apply(lambda x: max(0, x - day))

    return df[['Product', 'Effect_on_Nymphs', 'Effect_on_Adults', 'Available_in', 'Available_until_Harvest']]

# Example usage of the function, with dummy values for day, nymph_population, and adult_population
# Replace these with actual values as needed
example_day = 10
example_nymph_population = 100
example_adult_population = 50

# Call the function with the example values
result_df = insecticide_effectiveness(df, example_day, example_nymph_population, example_adult_population)
result_df.head()  # Displaying the first few rows of the resulting DataFrame

    

,Product,Effect_on_Nymphs,Effect_on_Adults,Available_in,Available_until_Harvest
0,Brigade,100.0,50.0,20.0,20.0
1,Sniper,100.0,50.0,20.0,20.0
2,Mustang Maxx,75.0,50.0,0.0,0.0
3,Baythroid,100.0,50.0,0.0,0.0
4,Danitol,100.0,50.0,11.0,11.0


In [12]:
def pesticide_application_strategy(temperature, precipitation, harvest_day, df):
    """
    Determines an optimal pesticide application strategy while considering environmental factors,
    the observed SLF population, and pesticide usage constraints.

    Parameters:
    temperature (list): Daily average temperatures.
    precipitation (list): Daily precipitation levels.
    harvest_day (int): The day of the year on which harvest is planned.

    Returns:
    DataFrame: A recommended pesticide application schedule.
    """
    # Define the total number of days in the season
    season_length = len(temperature)
    
    #get slf_proportion
    slf_proportion = slf_population_model(temperature, precipitation)
    
    # Initialize the application schedule DataFrame
    application_schedule = pd.DataFrame({
        'day': range(1, season_length + 1),
        'temperature': temperature,
        'precipitation': precipitation,
        
        """
        TODO: Address the total_count problem!!! Count is not the same as proportion
        """
        
        'nymph_count': slf_proportion['nymph_proportion'],
        'adult_count': slf_proportion['adult_proportion'],
        'pesticide': [None] * season_length,  # Placeholder for the chosen pesticide
        'application_rate': [0] * season_length  # Placeholder for the application rate
    })

    # Initialize a dictionary to track seasonal usage of each pesticide
    last_use_day = {pesticide_class: -np.inf for pesticide_class in df['Class'].unique()}

    seasonal_usage = {product: 0 for product in df['Product']}
    
    # Iterate through each day in the season
    
    for index, row in application_schedule.iterrows():
        pesticide_applied = False  # This should be set to False at the start of each iteration
        if row['nymph_count'] >= 15 or row['adult_count'] >= 5:
            for i, pesticide in df.iterrows():
                time_since_last_use = row['day'] - last_use_day[pesticide['Class']]
                if (pesticide['Effect on Adults'] >= 0.75 or pesticide['Effect on Nymphs'] >= 0.75) and \
                   time_since_last_use > 30 and \
                   seasonal_usage[pesticide['Product']] < pesticide['Seasonal Max']:
                    chosen_pesticide = pesticide['Product']
                    application_schedule.loc[index, 'pesticide'] = chosen_pesticide
                    
                    # Apply the pesticide if we are not within the PHI period before harvest
                    if row['day'] <= harvest_day - pesticide['PHI (days)']:
                        effectiveness_df = insecticide_effectiveness(
                            df, 
                            row['day'], 
                            row['nymph_count'], 
                            row['adult_count']
                        )
                        application_rate = effectiveness_df.loc[
                            effectiveness_df['Product'] == chosen_pesticide, 'Effect_on_Nymphs'
                        ].iloc[0]
                        application_schedule.loc[index, 'application_rate'] = application_rate

                        seasonal_usage[chosen_pesticide] += application_rate
                        last_use_day[pesticide['Class']] = row['day']
                        pesticide_applied = True  # Only set this to True if a pesticide is successfully applied

                        # Check if we have reached the Seasonal Max for the chosen pesticide
                        if seasonal_usage[chosen_pesticide] > df.loc[
                            df['Product'] == chosen_pesticide, 'Seasonal Max'
                        ].iloc[0]:
                            raise ValueError(f"Seasonal Max exceeded for chosen pesticide {chosen_pesticide}.")

                        break  # Break out of the loop after applying a pesticide

            # Raise an error if no pesticide could be applied due to PHI constraints or other issues
            if not pesticide_applied:
                raise ValueError(f"No suitable pesticide found or PHI constraints prevent application on day {row['day']}.")

    # Filter out any days where pesticide application is not possible due to PHI constraints
    application_schedule = application_schedule[application_schedule['day'] <= harvest_day - df['PHI (days)'].max()]

    return application_schedule

# Example usage
# These are placeholder values for temperatures and precipitation throughout the season.
temperature = [25] * 183  # Constant temperature for simplicity
precipitation = [5] * 183  # Constant precipitation for simplicity

# Placeholder values for the observed SLF population.
# In practice, these would come from actual observations or a population model.
slf_observations = {
    'nymph_proportion': np.random.rand(183),  # Randomly generated for example purposes
    'adult_proportion': np.random.rand(183)   # Randomly generated for example purposes
}

harvest_day = 160  # Placeholder value for the day of harvest

# Assuming we've already loaded pesticide_data from a CSV file
# The 'Seasonal Max' would need to be added to the DataFrame based on your pesticide data
# Generate the application schedule

application_schedule = pesticide_application_strategy(temperature, precipitation, harvest_day, df)
print(application_schedule)


     day  temperature  precipitation  nymph_count  adult_count pesticide  \
0      1           25              5          0.5          0.0      None   
1      2           25              5          0.5          0.0      None   
2      3           25              5          0.5          0.0      None   
3      4           25              5          0.5          0.0      None   
4      5           25              5          0.5          0.0      None   
..   ...          ...            ...          ...          ...       ...   
125  126           25              5          0.0          0.5      None   
126  127           25              5          0.0          0.5      None   
127  128           25              5          0.0          0.5      None   
128  129           25              5          0.0          0.5      None   
129  130           25              5          0.0          0.5      None   

     application_rate  
0                   0  
1                   0  
2              